In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image #Import Pillow for image manipulation
import gc #Import gc for garbage collector

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
#Create dataframe based on th csv file
df_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')

#Change the names of the columns to ones more significant
new_column_names = {
    'isic_id': 'Unique_case_identifier',
    'target': 'Target',
    'age_approx': 'Approximate_age',
    'sex': 'Patient_sex',
    'anatom_site_general': 'General_anatomical_site',
    'clin_size_long_diam_mm': 'Longest_diameter_mm',
    'image_type': 'Image_type',
    'tbp_tile_type': 'Lighting_modality_3D_TBP',
    'tbp_lv_A': 'A_inside_lesion',
    'tbp_lv_Aext': 'A_outside_lesion',
    'tbp_lv_B': 'B_inside_lesion',
    'tbp_lv_Bext': 'B_outside_lesion',
    'tbp_lv_C': 'C_inside_lesion',
    'tbp_lv_Cext': 'C_outside_lesion',
    'tbp_lv_H': 'Hue_inside_lesion',
    'tbp_lv_Hext': 'Hue_outside_lesion',
    'tbp_lv_L': 'L_inside_lesion',
    'tbp_lv_Lext': 'L_outside_lesion',
    'tbp_lv_areaMM2': 'Lesion_area_mm2',
    'tbp_lv_area_perim_ratio': 'Border_jaggedness_ratio',
    'tbp_lv_color_std_mean': 'Color_irregularity_mean',
    'tbp_lv_deltaA': 'Average_A_contrast',
    'tbp_lv_deltaB': 'Average_B_contrast',
    'tbp_lv_deltaL': 'Average_L_contrast',
    'tbp_lv_deltaLB': 'Average_delta_LB',
    'tbp_lv_deltaLBnorm': 'Contrast_lesion_surrounding_skin',
    'tbp_lv_eccentricity': 'Lesion_eccentricity',
    'tbp_lv_location': 'Anatomical_location',
    'tbp_lv_location_simple': 'Simple_anatomical_location',
    'tbp_lv_minorAxisMM': 'Smallest_diameter_mm',
    'tbp_lv_nevi_confidence': 'Nevus_confidence_score',
    'tbp_lv_norm_border': 'Border_irregularity_normalized',
    'tbp_lv_norm_color': 'Color_variation_normalized',
    'tbp_lv_perimeterMM': 'Lesion_perimeter_mm',
    'tbp_lv_radial_color_std_max': 'Color_asymmetry',
    'tbp_lv_stdL': 'Standard_deviation_L_inside',
    'tbp_lv_Lext': 'Standard_deviation_L_outside',
    'tbp_lv_symm_2axis': 'Border_asymmetry',
    'tbp_lv_symm_2axis_angle': 'Border_asymmetry_angle',
    'tbp_lv_x': 'Lesion_X_coordinate',
    'tbp_lv_y': 'Lesion_Y_coordinate',
    'tbp_lv_z': 'Lesion_Z_coordinate',
    'attribution': 'Image_attribution',
    'license': 'Image_license',
    'lesion_id': 'Unique_lesion_identifier',
    'iddx_full': 'Full_diagnosis_code',
    'iddx_1': 'Primary_diagnosis_category',
    'iddx_2': 'Secondary_diagnosis_category',
    'iddx_3': 'Tertiary_diagnosis_category',
    'iddx_4': 'Quaternary_diagnosis_category',
    'iddx_5': 'Quinary_diagnosis_category',
    'mel_mitotic_index': 'Melanoma_mitotic_index',
    'mel_thick_mm': 'Melanoma_thickness_mm',
    'tbp_lv_dnn_lesion_confidence': 'DNN_lesion_confidence_score'
}

df_metadata.rename(columns=new_column_names, inplace=True)

#Delate columns with no information
df_limp_columns = df_metadata.drop(columns=["Unique_lesion_identifier","Secondary_diagnosis_category","Tertiary_diagnosis_category","Quaternary_diagnosis_category","Quinary_diagnosis_category","Melanoma_mitotic_index","Melanoma_thickness_mm"])

#Deleate useless columns
df_limp = df_limp_columns.drop(columns=["copyright_license","Image_attribution","Image_type", "Lighting_modality_3D_TBP","Anatomical_location","Simple_anatomical_location"])

#Create nulls based on registers
df_limp.replace(['NaN', 'na', 'Na', 'n/a', 'N/A', 'NA', 'nan'], np.nan, inplace=True)

#Clear registers with null values in the columns that contain less than 10%
columns_to_check = ["Approximate_age","Patient_sex","General_anatomical_site", "Unique_case_identifier"]
df_clean = df_limp.dropna(subset=columns_to_check)

#We should drop any redundant unique identifier and redundant diagnosis
df_clean = df_clean.drop(columns= ["patient_id","Full_diagnosis_code"])

#Encode each variable
df_encoded = pd.get_dummies(df_clean, columns=['Patient_sex','General_anatomical_site','Primary_diagnosis_category'], prefix='Cat')

df_encoded.info()

/tmp/ipykernel_34/2902822491.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


<class 'pandas.core.frame.DataFrame'>
Index: 381914 entries, 0 to 401058
Data columns (total 47 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Unique_case_identifier            381914 non-null  object 
 1   Target                            381914 non-null  int64  
 2   Approximate_age                   381914 non-null  float64
 3   Longest_diameter_mm               381914 non-null  float64
 4   A_inside_lesion                   381914 non-null  float64
 5   A_outside_lesion                  381914 non-null  float64
 6   B_inside_lesion                   381914 non-null  float64
 7   B_outside_lesion                  381914 non-null  float64
 8   C_inside_lesion                   381914 non-null  float64
 9   C_outside_lesion                  381914 non-null  float64
 10  Hue_inside_lesion                 381914 non-null  float64
 11  Hue_outside_lesion                381914 non-null  float6

# Data Normalization

We use a StandartScaler based on the normal distribution of some color data, and the amount of 0 or outlairs is not a considerable amount

In [3]:
from sklearn.preprocessing import StandardScaler

#Separeate numeric cols from unique identifier for normalization
numeric_cols = ['Approximate_age', 'Longest_diameter_mm', 'A_inside_lesion',
       'A_outside_lesion', 'B_inside_lesion', 'B_outside_lesion',
       'C_inside_lesion', 'C_outside_lesion', 'Hue_inside_lesion',
       'Hue_outside_lesion', 'L_inside_lesion', 'Standard_deviation_L_outside',
       'Lesion_area_mm2', 'Border_jaggedness_ratio', 'Color_irregularity_mean',
       'Average_A_contrast', 'Average_B_contrast', 'Average_L_contrast',
       'Average_delta_LB', 'Contrast_lesion_surrounding_skin',
       'Lesion_eccentricity', 'Smallest_diameter_mm', 'Nevus_confidence_score',
       'Border_irregularity_normalized', 'Color_variation_normalized',
       'Lesion_perimeter_mm', 'Color_asymmetry', 'Standard_deviation_L_inside',
       'tbp_lv_stdLExt', 'Border_asymmetry', 'Border_asymmetry_angle',
       'Lesion_X_coordinate', 'Lesion_Y_coordinate', 'Lesion_Z_coordinate',
       'DNN_lesion_confidence_score']
non_numeric_cols = df_encoded.select_dtypes(exclude=['number']).columns

#Scale with standard Scaler
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_encoded[numeric_cols]), columns=numeric_cols)

#Merge both dataframes 
df_final = pd.concat([df_normalized,df_encoded[non_numeric_cols]], axis=1)
df_final = pd.concat([df_final, df_encoded[['Target']]], axis=1)

#Convert Boolean values to integer
bool_cols = ['Cat_female','Cat_male','Cat_anterior torso','Cat_head/neck','Cat_lower extremity','Cat_posterior torso','Cat_upper extremity','Cat_Benign','Cat_Indeterminate','Cat_Malignant']
df_final[bool_cols] = df_encoded[bool_cols].astype(int)

#Eliminar valores nulos nuevamente
df_final_limp = df_final.dropna()

df_final_limp.head(5)

,Approximate_age,Longest_diameter_mm,A_inside_lesion,A_outside_lesion,B_inside_lesion,B_outside_lesion,C_inside_lesion,C_outside_lesion,Hue_inside_lesion,Hue_outside_lesion,...,Cat_male,Cat_anterior torso,Cat_head/neck,Cat_lower extremity,Cat_posterior torso,Cat_upper extremity,Cat_Benign,Cat_Indeterminate,Cat_Malignant,Target
0,0.146827,-0.511968,0.074082,0.385571,-0.238691,-0.642496,-0.175968,-0.390812,-0.277032,-0.904842,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.146827,-1.623306,2.952957,2.977714,-0.351795,-0.508521,1.155279,0.936583,-2.700177,-2.995601,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.146827,-0.305740,0.659341,0.632232,1.874046,1.505248,1.677554,1.420286,0.849175,0.353501,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.512630,-0.408854,-1.432637,-0.781171,-1.285554,-1.281011,-1.578606,-1.348601,0.331834,-0.152861,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.218976,-0.689553,1.198982,1.466393,-0.326189,-0.246849,0.271647,0.373818,-1.386033,-1.577969,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Normalization of Images in train-image directory

In [4]:
"""
import cv2

def obtener_info_imagen(path:str) -> dict:
    ret = {}
    img = cv2.imread(path)
    alto, ancho, canales = img.shape
    ret['Alto'] = alto
    ret['Ancho'] = ancho
    return ret

proto_df = []
img_list = df_final_limp["Unique_case_identifier"].tolist()
for img_path in img_list:
    reporte = obtener_info_imagen(f"/kaggle/input/isic-2024-challenge/train-image/image/{img_path}.jpg")
    if reporte is not None:
        proto_df.append(reporte)
df = pd.DataFrame(proto_df)
df.describe()
"""

'\nimport cv2\n\ndef obtener_info_imagen(path:str) -> dict:\n    ret = {}\n    img = cv2.imread(path)\n    alto, ancho, canales = img.shape\n    ret[\'Alto\'] = alto\n    ret[\'Ancho\'] = ancho\n    return ret\n\nproto_df = []\nimg_list = df_final_limp["Unique_case_identifier"].tolist()\nfor img_path in img_list:\n    reporte = obtener_info_imagen(f"/kaggle/input/isic-2024-challenge/train-image/image/{img_path}.jpg")\n    if reporte is not None:\n        proto_df.append(reporte)\ndf = pd.DataFrame(proto_df)\ndf.describe()\n'

The point of the commented code was to make an small exploration of the data already filtered to determine the sizes of the images still present. 

We concluded that the **max size is 269x269** and the **min size is 41x41**

Taking this into account,the resize normalization we apply is for **224x224**

**Important**
As in kaggle enviroment RAM usage is limited and the access to the machine files is only in read mode, to process the images correctly and passing them to the Deep Learning model, we came out with the following solution:

1. **Load a Batch into Memory**: Load only a single batch of images into memory at a time.
2. **Preprocess the Batch**: Resize and normalize the images.
3. **Train on the Batch**: Feed the batch to the CNN model for training.
4. **Clear Memory**: Delete the batch from memory to free up space.
5. **Repeat**: Continue this process until all data is processed.

# Train - Test - Validate Split 

In [4]:
from sklearn.model_selection import train_test_split

#Create dataframes split -> Train 80%, Test 10%, Validation 10%

train_df, test_df = train_test_split(df_final_limp, test_size=0.2, random_state=42)

print("Training Set:\n", len(train_df))
print("\nTest Set:\n", len(test_df))

Training Set:
 290943

Test Set:
 72736


# NN (Deep Neuronal Network + Convolutional NN) Architecture

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate

# First input for 44 neurons (Additional Information of the image in dataset) (Branch-1)
numeric_input = Input(shape=(44,), name='numeric_input')

# Hidden layers for the 44-neuron input (Branch-1)
numeric_1 = Dense(64, activation='relu')(numeric_input)
numeric_2 = Dense(32, activation='relu')(numeric_1)

# Input for the image of 224x224 (Branch-2)
image_input = Input(shape=(224, 224, 3), name='image_input')

# CNN layers for image processing (Branch-2)
convolution_image_1 = Conv2D(32, (3, 3), activation='relu')(image_input)
pool_image_1 = MaxPooling2D((2, 2))(convolution_image_1)
convolution_image_2 = Conv2D(64, (3, 3), activation='relu')(pool_image_1)
pool_image_2 = MaxPooling2D((2, 2))(convolution_image_2)
convolution_image_3 = Conv2D(128, (3, 3), activation='relu')(pool_image_2)
pool_image_3 = MaxPooling2D((2, 2))(convolution_image_3)
flatten_layer = Flatten()(pool_image_3)

# Concatenate in one tensor both branches
combined = Concatenate()([numeric_2, flatten_layer])

# Dense layers after concatenation for further interpretation
dense_1 = Dense(64, activation='relu')(combined)
dense_2 = Dense(32, activation='relu')(dense_1)

# Output layer with 1 nodes (probabilities of a 1)
output = Dense(1, activation='sigmoid')(dense_2)

# Define the model 
model = Model(inputs=[numeric_input, image_input], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

2024-08-31 03:41:09.143836: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 03:41:09.143983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 03:41:09.301370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        896 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 111, 111,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 54, 54,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_input       │ (None, 44)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 52, 52,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      2,880 │ numeric_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 26, 26,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 86528)     │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 86560)     │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │  5,539,904 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │         33 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,640,225 (21.52 MB)

 Trainable params: 5,640,225 (21.52 MB)

 Non-trainable params: 0 (0.00 B)

## Training of deep learning model via Batches

In [7]:
def normalize_image_directory(uci: str):
    path = f"/kaggle/input/isic-2024-challenge/train-image/image/{uci}.jpg"
    img = Image.open(path)
    img = img.resize((224, 224))
    img_array = np.array(img) #Convert image to array of numbers
    img_array = img_array / 255.0 #Perform normalization of data in the image
    del path, img
    return img_array

def batch_generator(train_df: pd.DataFrame, batch_size: int =32):
    excluded_columns = ["DNN_lesion_confidence_score", "Unique_case_identifier", "Target"]
    num_samples = len(train_df) #Total lenght of the indexes
    while True: #Use infinite loop since the generator uses yield (Waits to the next interation in an inifinite fashion)
        for start_idx in range(0, num_samples, batch_size): #Iterate throught the dataset in size batch_size
            end_idx = min(start_idx + batch_size, num_samples) #Use min function to not call a out of index
            batch_df = train_df.iloc[start_idx:end_idx] #Select the appropiate slice
            
            #Initialize batch variables
            images = []
            registers = []
            labels = []
            for _, row in batch_df.iterrows(): #Iterate through the batch dataset and extract
                img_array = normalize_image_directory(row["Unique_case_identifier"])
                register_data = row.drop(labels=excluded_columns).to_numpy(dtype=np.float32)
                label = row["Target"]
                
                images.append(img_array)
                registers.append(register_data)
                labels.append(label)
            
            yield ([np.array(registers), np.array(images)], np.array(labels))
            
            # Clear memory for limited resources
            del images, registers, labels
            gc.collect() #Call garbage collector



def fit_model(model, train_df, batch_size, epochs):
    steps_per_epoch = len(train_df) // batch_size
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        epoch_loss = []
        epoch_accuracy = []
        
        generator = batch_generator(train_df, batch_size)
        
        for step in range(steps_per_epoch):
            try:
                inputs, labels = next(generator)
                result = model.train_on_batch(inputs, labels)
                
                epoch_loss.append(result[0])
                epoch_accuracy.append(result[1])
                
                if (step + 1) % 10 == 0:
                    print(f"Batch {step+1}/{steps_per_epoch} - Loss: {np.mean(epoch_loss):.4f}, Accuracy: {np.mean(epoch_accuracy):.4f}")
                
                # Clear memory
                del inputs, labels
                gc.collect()
                
            except Exception as e:
                print(f"Error during training on batch: {e}")
                return
        
        print(f"Epoch {epoch+1}/{epochs} - Loss: {np.mean(epoch_loss):.4f}, Accuracy: {np.mean(epoch_accuracy):.4f}")
        


In [ ]:
#Variables for fitting the model
batch_size = 32
epochs = 100

fit_model(model, train_df, batch_size, epochs)

Epoch 1/100
Batch 10/9091 - Loss: 0.1874, Accuracy: 1.0000
Batch 20/9091 - Loss: 0.2143, Accuracy: 0.9995
Batch 30/9091 - Loss: 0.2554, Accuracy: 0.9988
Batch 40/9091 - Loss: 0.2521, Accuracy: 0.9987
Batch 50/9091 - Loss: 0.2395, Accuracy: 0.9987
Batch 60/9091 - Loss: 0.2253, Accuracy: 0.9987
Batch 70/9091 - Loss: 0.2118, Accuracy: 0.9987
Batch 80/9091 - Loss: 0.1995, Accuracy: 0.9988
Batch 90/9091 - Loss: 0.1885, Accuracy: 0.9988
Batch 100/9091 - Loss: 0.1786, Accuracy: 0.9989
Batch 110/9091 - Loss: 0.1697, Accuracy: 0.9989
Batch 120/9091 - Loss: 0.1622, Accuracy: 0.9990


**Result**: Since the model is really large, and the dataset is even larger, it was predictable that a solution that uses all the evidence available would be incapable of training given the resources of Kaggle, despide efforts to optimice the trainig with custom generators, batch training, and custom fit model functions. 
Even if the best way is to use all the evidence (Because in the transformation analysis we determined that all variables provide different solutions), it is a non-viable solution.

**Posible solutions**:
1. Try the same approach with a more compressed image (Eliminating important details) and reducing the dense branch neurons (Eliminating features that may not be as important)
2. Creating two solutions (Each per branch) and evaluate the performance in a individual way

# Reduced Mixed NN (Deep Neuronal Network + Convolutional NN) Architecture

There are some features that will change to try to train the NN based on the feature of a mixed aproach. We will sacrifice detail for memmory (Resource) efficiency, this by dropping some important features and decreasing the number of neurons requiered for the network to exist. **Changes**:

1. Decrease the size of the traget image to (67, 67)
2. Select only key features based on the Correlation Heatmap and resarch of the context (Stadistical mesurments of colors, symmetry of the lesion, size of the lesion and categorical variables)

In [9]:
train_df.columns

Index(['Approximate_age', 'Longest_diameter_mm', 'A_inside_lesion',
       'A_outside_lesion', 'B_inside_lesion', 'B_outside_lesion',
       'C_inside_lesion', 'C_outside_lesion', 'Hue_inside_lesion',
       'Hue_outside_lesion', 'L_inside_lesion', 'Standard_deviation_L_outside',
       'Lesion_area_mm2', 'Border_jaggedness_ratio', 'Color_irregularity_mean',
       'Average_A_contrast', 'Average_B_contrast', 'Average_L_contrast',
       'Average_delta_LB', 'Contrast_lesion_surrounding_skin',
       'Lesion_eccentricity', 'Smallest_diameter_mm', 'Nevus_confidence_score',
       'Border_irregularity_normalized', 'Color_variation_normalized',
       'Lesion_perimeter_mm', 'Color_asymmetry', 'Standard_deviation_L_inside',
       'tbp_lv_stdLExt', 'Border_asymmetry', 'Border_asymmetry_angle',
       'Lesion_X_coordinate', 'Lesion_Y_coordinate', 'Lesion_Z_coordinate',
       'DNN_lesion_confidence_score', 'Unique_case_identifier', 'Cat_female',
       'Cat_male', 'Cat_anterior torso', 'Cat_h

Feature Columns selected: ['Hue_inside_lesion','Hue_outside_lesion','Lesion_area_mm2', 'Border_jaggedness_ratio','Color_irregularity_mean','Contrast_lesion_surrounding_skin',
'Lesion_eccentricity','Border_irregularity_normalized','Color_variation_normalized',
'Color_asymmetry','Border_asymmetry',Cat_Benign', 'Cat_Indeterminate', 'Cat_Malignant']

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate

# First input for 14 neurons (Additional Information of the image in dataset) (Branch-1)
numeric_input_redux = Input(shape=(14,), name='numeric_input_redux')

# Hidden layers for the 14-neuron input (Branch-1)
numeric_1_redux = Dense(64, activation='relu')(numeric_input_redux)
numeric_2_redux = Dense(32, activation='relu')(numeric_1_redux)

# Input for the image of 224x224 (Branch-2)
image_input_redux = Input(shape=(67, 67, 3), name='image_input_redux')

# CNN layers for image processing (Branch-2)
convolution_image_redux = Conv2D(32, (3, 3), activation='relu')(image_input_redux)
pool_image_1_redux = MaxPooling2D((2, 2))(convolution_image_redux)
convolution_image_1_redux = Conv2D(64, (3, 3), activation='relu')(pool_image_1_redux)
pool_image_2_redux = MaxPooling2D((2, 2))(convolution_image_1_redux)
convolution_image_2_redux = Conv2D(128, (3, 3), activation='relu')(pool_image_2_redux)
pool_image_3_redux = MaxPooling2D((2, 2))(convolution_image_2_redux)
flatten_layer_redux = Flatten()(pool_image_3_redux)

# Concatenate in one tensor both branches
combined_redux = Concatenate()([numeric_2_redux, flatten_layer_redux])

# Dense layers after concatenation for further interpretation
dense_1_redux = Dense(64, activation='relu')(combined_redux)
dense_2_redux = Dense(32, activation='relu')(dense_1_redux)

# Output layer with 1 nodes (probabilities of a 1)
output_redux = Dense(1, activation='sigmoid')(dense_2_redux)

# Define the model 
model_redux = Model(inputs=[numeric_input_redux, image_input_redux], outputs=output_redux)

model_redux.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_redux.summary()

2024-08-31 07:03:49.559658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 07:03:49.559765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 07:03:49.682203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input_redux   │ (None, 67, 67, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 65, 65,    │        896 │ image_input_redu… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 32, 32,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 30, 30,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 15, 15,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_input_redux │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 13, 13,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        960 │ numeric_input_re… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 6, 6, 128) │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4608)      │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 4640)      │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │    297,024 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │         33 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 395,425 (1.51 MB)

 Trainable params: 395,425 (1.51 MB)

 Non-trainable params: 0 (0.00 B)

## Training of deep learning model via Batches

In [6]:
def normalize_image_directory_redux(uci: str):
    path = f"/kaggle/input/isic-2024-challenge/train-image/image/{uci}.jpg"
    img = Image.open(path)
    img = img.resize((67, 67))
    img_array = np.array(img) #Convert image to array of numbers
    img_array = img_array / 255.0 #Perform normalization of data in the image
    del path, img
    return img_array

def batch_generator_redux(train_df: pd.DataFrame, batch_size: int =32):
    excluded_columns = [
    'Approximate_age', 'Longest_diameter_mm', 'A_inside_lesion', 'A_outside_lesion', 'B_inside_lesion', 
    'B_outside_lesion', 'C_inside_lesion', 'C_outside_lesion', 'L_inside_lesion', 
    'Standard_deviation_L_outside', 'Average_A_contrast', 'Average_B_contrast', 'Average_L_contrast', 
    'Average_delta_LB', 'Smallest_diameter_mm', 'Nevus_confidence_score', 'Lesion_perimeter_mm', 
    'Standard_deviation_L_inside', 'tbp_lv_stdLExt', 'Border_asymmetry_angle', 'Lesion_X_coordinate', 
    'Lesion_Y_coordinate', 'Lesion_Z_coordinate', 'DNN_lesion_confidence_score', 'Unique_case_identifier', 
    'Cat_female', 'Cat_male', 'Cat_anterior torso', 'Cat_head/neck', 'Cat_lower extremity', 
    'Cat_posterior torso', 'Cat_upper extremity', 'Target']
    num_samples = len(train_df) #Total lenght of the indexes
    while True: #Use infinite loop since the generator uses yield (Waits to the next interation in an inifinite fashion)
        for start_idx in range(0, num_samples, batch_size): #Iterate throught the dataset in size batch_size
            end_idx = min(start_idx + batch_size, num_samples) #Use min function to not call a out of index
            batch_df = train_df.iloc[start_idx:end_idx] #Select the appropiate slice
            
            #Initialize batch variables
            images = []
            registers = []
            labels = []
            for _, row in batch_df.iterrows(): #Iterate through the batch dataset and extract
                img_array = normalize_image_directory_redux(row["Unique_case_identifier"])
                register_data = row.drop(labels=excluded_columns).to_numpy(dtype=np.float32)
                label = row["Target"]
                
                images.append(img_array)
                registers.append(register_data)
                labels.append(label)
            
            yield ([np.array(registers), np.array(images)], np.array(labels))
            
            # Clear memory for limited resources
            del images, registers, labels
            gc.collect() #Call garbage collector



def fit_model_redux(model, train_df, batch_size, epochs):
    steps_per_epoch = len(train_df) // batch_size
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        epoch_loss = []
        epoch_accuracy = []
        
        generator = batch_generator_redux(train_df, batch_size)
        
        for step in range(steps_per_epoch):
            try:
                inputs, labels = next(generator)
                result = model.train_on_batch(inputs, labels)
                
                epoch_loss.append(result[0])
                epoch_accuracy.append(result[1])
                
                if (step + 1) % 10 == 0:
                    print(f"Batch {step+1}/{steps_per_epoch} - Loss: {np.mean(epoch_loss):.4f}, Accuracy: {np.mean(epoch_accuracy):.4f}")
                
                # Clear memory
                del inputs, labels
                gc.collect()
                
            except Exception as e:
                print(f"Error during training on batch: {e}")
                return
        
        print(f"Epoch {epoch+1}/{epochs} - Loss: {np.mean(epoch_loss):.4f}, Accuracy: {np.mean(epoch_accuracy):.4f}")
        


In [ ]:
#Variables for fitting the model
batch_size = 32
epochs = 100

fit_model_redux(model_redux, train_df, batch_size, epochs)

Epoch 1/100


I0000 00:00:1725087858.789469     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1725087858.812150     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Batch 10/9091 - Loss: 0.3793, Accuracy: 0.7163
Batch 20/9091 - Loss: 0.2727, Accuracy: 0.8252
Batch 30/9091 - Loss: 0.2366, Accuracy: 0.8698
Batch 40/9091 - Loss: 0.2068, Accuracy: 0.8950
Batch 50/9091 - Loss: 0.1838, Accuracy: 0.9115
Batch 60/9091 - Loss: 0.1657, Accuracy: 0.9231
Batch 70/9091 - Loss: 0.1511, Accuracy: 0.9319
Batch 80/9091 - Loss: 0.1391, Accuracy: 0.9387
Batch 90/9091 - Loss: 0.1290, Accuracy: 0.9441
Batch 100/9091 - Loss: 0.1205, Accuracy: 0.9486
Batch 110/9091 - Loss: 0.1131, Accuracy: 0.9524
Batch 120/9091 - Loss: 0.1070, Accuracy: 0.9556
Batch 130/9091 - Loss: 0.1018, Accuracy: 0.9584
Batch 140/9091 - Loss: 0.0973, Accuracy: 0.9608
Batch 150/9091 - Loss: 0.0932, Accuracy: 0.9629
Batch 160/9091 - Loss: 0.0894, Accuracy: 0.9648
Batch 170/9091 - Loss: 0.0860, Accuracy: 0.9665
Batch 180/9091 - Loss: 0.0828, Accuracy: 0.9680
Batch 190/9091 - Loss: 0.0803, Accuracy: 0.9694
Batch 200/9091 - Loss: 0.0779, Accuracy: 0.9707
Batch 210/9091 - Loss: 0.0758, Accuracy: 0.9718
B

**Result**: Knowing that the reduction of sizes was of aproximatly more than 50% and that the improvment on batch processing was only of 8.8%, we can conclude that this aproach, despide of the promessing metrics is non-viable with the resources provided.